In [ ]:
from database.connect import getConnection
import mysql
import pandas as pd
from pathlib import Path
import csv

try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

Runtime W df method: 5.4 s  
Runtime W str method: 5.7 s

In [ ]:
file_path = Path("./pre-processed/wordcounts.csv")

# check if the file exists
my_file = file_path
if not my_file.is_file():
    # create the file if it doesn't exist
    with open(file_path, "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["word", "count"])

    word_counts = pd.DataFrame(columns=["word", "count"])

    # get the text from the database
    cursor.execute("SELECT text FROM tweets WHERE text not like 'RT%'")
    tweets = cursor.fetchall()
    print('got all tweets')

    word_counts = {}
    for tweet in tweets:
        for word in tweet[0].split():
            word = word.lower().strip(".,:;!?\"\'()[]{}")
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1
    
    # save the sorted word counts to the file
    with open(file_path, "a", newline="") as csvfile:
        writer = csv.writer(csvfile)
        for word, count in sorted(word_counts.items(), key=lambda x: x[1], reverse=True):
            writer.writerow([word, count])

In [ ]:
# read the file
word_counts = pd.read_csv(file_path)

# get the top 100 words
top_words = word_counts.groupby("word").sum().sort_values(by="count", ascending=False).head(100)
top_words